In [ ]:
# 필요한 라이브러리를 불러온다

from openai import OpenAI
from PIL import Image

import base64
import openai
import pillow_heif
import pandas as pd
import os

In [37]:
# OpenAI 버전 확인
print(openai.__version__)

1.54.4


In [ ]:
os.environ["OPENAI_API_KEY"] = "." # -> OpenAI API key 입력
client = OpenAI()

## HEIC를 JPG로 변환 후 사진에 대한 텍스트 묘사 출력

In [ ]:
class Image2Text:
    # 생성자 함수
    def __init__(self, heic_path, output_dir):
        self.heic_path = heic_path                      # HEIC 이미지 경로
        self.output_dir = output_dir                    # JPG로 변환된 이미지를 저장할 경로
        self.jpg_path = None

    # HEIC 확장자를 JPG 확장자로 변환
    def convert_heic_to_jpg(self):
        heif_file = pillow_heif.open_heif(self.heic_path) # pillow_heif 라이브러리를 사용하여 HEIC 이미지를 열기
        
        # HEIC 이미지를 PIL 이미지로 변환하는 이유: PIL 이미지는 다양한 형식으로 저장할 수 있다. 
        # 예를 들어, HEIC 이미지를 JPG, PNG, BMP 등 다른 형식으로 변환하여 저장할 수 있다. 
        # 이는 호환성을 높이고, 특정 형식만 지원하는 시스템이나 애플리케이션에서 이미지를 사용할 수 있다.
        
        image = Image.frombytes(                        # HEIC 이미지를 PIL 이미지로 변환
            heif_file.mode,                             # 이미지 모드, 예: RGB -> 컬러 이미지를 의미
            heif_file.size,                             # 이미지 크기, 예: (1920, 1080) -> 1920x1080 크기의 이미지를 의미
            heif_file.data,                             # 이미지 데이터를 바이트 형태로 나타낸 것
            "raw",                                      # 원시 데이터 형식을 나타내는 문자열
            heif_file.mode,                             # 이미지 모드, 예: RGB -> 컬러 이미지를 의미
            heif_file.stride,                           # 이미지의 각 행의 바이트 수
        )                                               # HEIC 이미지를 PIL 이미지로 변환
        
        base_name = os.path.basename(self.heic_path)    # 파일 경로에서 파일 이름만 추출
        file_name, _ = os.path.splitext(base_name)      # 파일 이름과 확장자(heic)를 분리
        self.jpg_path = os.path.join(self.output_dir, f"{file_name}.jpg") # jpg로 변환된 이미지를 저장할 경로
        image.save(self.jpg_path, "JPEG")               # jpg로 변환된 이미지 파일을 output_dir 경로에 저장

    # 이미지를 JPG로 변환하고 base64로 인코딩하여 반환
    # 인코딩을 하는 이유: OpenAI API에 이미지를 전송할 때 Base64 인코딩을 사용해야 이미지를 텍스트 데이터로 전송할 수 있기 때문이다.
    def convert_and_encode_image(self):
        self.convert_heic_to_jpg()
        with open(self.jpg_path, "rb") as image_file:  # jpg_path 
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 이미지에 대한 질문을 OpenAI API에 요청 후 응답 반환
    def ask_about_image(self, client):
        base64_image = self.convert_and_encode_image()
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # -> gpt-4o-mini 모델 사용
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": (
                                '이 사진에 나와있는 음식은 무엇인가요? '
                                '음식 이름에는 따옴표("")를 붙여서 답변해 주세요. '
                                '또한 이 음식의 특징도 알려주세요. '
                                '예시: 이 사진에 나와 있는 음식은 "닭갈비"입니다.\n\n'
                                '"닭갈비"의 특징은 다음과 같습니다:\n'
                                '- 주재료로는 닭고기가 사용되며, 일반적으로 매운 양념에 재워서 조리합니다.\n'
                                '- 양배추, 고구마, 떡, 면 등 다양한 채소와 곁들어져 함께 볶아집니다.\n'
                                '- 일반적으로 철판이나 팬에서 함께 조리하며, 뜨겁게 먹는 것이 특징입니다.\n'
                                '- 찬바람에 대응해 매운 맛과 함께 따뜻하고 푸짐한 식사를 제공합니다.'
                            ),
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            },
                        },
                    ],
                }
            ],
        )
        return response.choices[0].message.content
    
    # 응답 내용에서 음식 이름 추출
    def extract_food_name(self, response_content):
        # 응답 내용을 문장으로 분할
        sentences = response_content.split('.')
        # 따옴표가 포함된 단어를 찾음
        for sentence in sentences:
            if '"' in sentence:
                # 따옴표로 감싸진 음식 이름 추출
                food_name = sentence.split('"')[1]
                return food_name
        return None
    
    # 응답 내용과 음식 이름을 데이터프레임에 저장하고 기록
    def save_to_dataframe(self, response_content, food_name, csv_path):
        data = {
            'response_content': [response_content],
            'food_name': [food_name]
        }
        df = pd.DataFrame(data)
        if not os.path.isfile(csv_path):
            df.to_csv(csv_path, index=False)
        else:
            df.to_csv(csv_path, mode='a', header=False, index=False)
            

output_dir = "C:/Users/KimDongyoung/Desktop"             # -> JPG로 변환된 이미지를 저장할 경로 입력
csv_path = "C:/Users/KimDongyoung/Desktop/food_names.csv" # -> CSV 파일 경로 입력


이 사진에 나와 있는 음식은 "비빔당면"입니다.

"비빔당면"의 특징은 다음과 같습니다:
- 주재료로는 당면(고구마 전분으로 만든 면류)과 다양한 채소가 사용됩니다.
- 보통 고추장 소스를 사용해 버무리며, 매콤하고 새콤한 맛이 특징입니다.
- 삶은 계란과 여러 가지 채소(상추, 오이, 미역 등)를 곁들여 제공합니다.
- 비빔당면은 차가운 면 요리로, 여름철에 특히 인기가 있습니다.
음식 이름: 비빔당면


In [ ]:
# Define paths for the three HEIC images
heic_paths = [
    "C:/Users/KimDongyoung/Desktop/IMG_6409.HEIC",
    "C:/Users/KimDongyoung/Desktop/IMG_6410.HEIC",
    # "C:/Users/KimDongyoung/Desktop/IMG_6412.HEIC"
]

# Loop through each HEIC image, convert, and process it
for heic_path in heic_paths:
    converter = Image2Text(heic_path, output_dir)
    response_content = converter.ask_about_image(client)
    print(response_content)
    
    # Extract the food name from the response content
    food_name = converter.extract_food_name(response_content)
    print(f"음식 이름: {food_name}")
    
    # Save the response content and food name to a CSV file
    converter.save_to_dataframe(response_content, food_name, csv_path)

이 사진에 나와 있는 음식은 "제육볶음"입니다.

"제육볶음"의 특징은 다음과 같습니다:
- 주재료로는 돼지고기가 사용되며, 고추장이나 고춧가루로 매콤하게 양념합니다.
- 양배추, 고추, 양파 등 다양한 채소와 함께 볶아내어 풍미를 더합니다.
- 일반적으로 밥과 함께 먹는 것이 일반적이며, 고소하고 매운 맛이 특징입니다.
- 전통적인 한국 가정식으로 사랑받으며, 맥주와 함께 안주로도 자주 즐겨집니다.
음식 이름: 제육볶음
이 사진에 나와 있는 음식은 "비빔냉면"입니다.

"비빔냉면"의 특징은 다음과 같습니다:
- 주재료로는 메밀면 또는 감자 전분 면이 사용되며, 차갑게 제공됩니다.
- 고추장이나 고춧가루를 사용한 매콤한 양념으로 비비는 것이 특징입니다.
- 시금치, 오이, 계란, 김 등의 다양한 채소와 함께 제공됩니다.
- 일반적으로 여름철 시원하게 즐기는 음식으로, 매콤함과 아삭한 식감이 어우러져 있습니다.
음식 이름: 비빔냉면


## 음식에 대한 정보를 csv 형식으로 저장

In [ ]:
# # Load the CSV file into a DataFrame
# df = pd.read_csv(csv_path)
# df

,response_content,food_name
0,"이 사진에 나와 있는 음식은 ""제육볶음""입니다.\n\n""제육볶음""의 특징은 다음과 ...",제육볶음
1,"이 사진에 나와 있는 음식은 ""비빔냉면""입니다.\n\n""비빔냉면""의 특징은 다음과 ...",비빔냉면
